# Project Title
## Data Engineering Capstone Project

### Project Summary

This project aims to unveil information about immigration into the United States. How does the flow of immigrants change seasonally, and what is their age profile? Where do they come from, and which cities are they traveling to? 

To accomplish this study, the following dataset have been used: 

- **I94 Immigration Data**: [comes from the U.S. National Tourism and Trade Office](https://travel.trade.gov/research/reports/i94/historical/2016.html) and contains various statistics on international visitor arrival in USA and comes from the US National Tourism and Trade Office. The dataset contains data from 2016.


- **Temperature Data**: comes from [Kaggle](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data) and is provided in a csv format, and includes information like average temperatur, date, and city.


- **Demographic Data:** comes from [OpenSoft](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/) and contains information about the demographics of all US cities such as race, male and female population, and median age.

#### Want to find out:

- What cities in the U.S. are the most common for arriving immigrants?
- How does immigration change throughout a year?
- Where do immigrants come from?
- What was the temperature difference in departure and arrival location?
- What is the gender ratio for the imigrants?
- What is the age profile the immigrants?
- What is the legal status of immigrants in the U.S.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [207]:
# Imports and installs 
import pandas as pd
import re
import glob

from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, udf
from pyspark.sql.types import StringType, IntegerType
from IPython.display import display
from IPython.core.display import HTML


# shows every column
pd.set_option('display.max_columns', 35)

## Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

[Why use Spark over Pandas](https://www.indellient.com/blog/a-journey-from-pandas-to-spark-data-frames/)

[1](https://github.com/the1schwartz/DEND-capstone/blob/master/City%20Immigration%20Demographics%20and%20Temperatures.ipynb)

[2](https://github.com/cheuklau/udacity-capstone/blob/master/Capstone%20Project%20Template.ipynb)

[3](https://github.com/nadirl00/Data-Engineering-Capstone-Project/blob/master/Capstone%20Project%20Template.ipynb)

In [208]:
spark = SparkSession.builder.\
    config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
    .enableHiveSupport()\
    .getOrCreate()

### Reading in Immigration Data to pandas

In [156]:
# read in the sample csv to pandas for easy assessing
pd_immi_df = pd.read_csv('immigration_data_sample.csv')
print('Finished reading Immigration csv file to pandas')

Finished reading Immigration csv file to pandas


In [157]:
# first look at the Immigration Data
pd_immi_df.head(3)

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,NaN,NaN,G,O,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,NaN,G,R,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,NaN,NaN,G,O,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT


### Reading in Immigration Data to Spark
This is to take advantage of the parallelization of spark, which pandas don't have. 

In [158]:
# reading in either the full immigration data or a sample to spark

# full imigration file, only use with EMR cluster
# big_file = glob.glob("../../data/18-83510-I94-Data-2016/*.sas7bdat")

# sample immigration file, only for April 2016
small_file = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'

# read in the data to Spark
spark_immi_df =spark.read.format('com.github.saurfang.sas.spark').load(small_file)

print('Finished reading Immigration sas.sample files to Spark.')

Finished reading Immigration sas.sample files to Spark.


In [159]:
# comparing with the spark Immigration Data
spark_immi_df.limit(3).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,None,NaN,37.0,2.0,1.0,None,None,None,T,None,U,None,1979.0,10282016,None,None,None,1.897628e+09,None,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,25.0,3.0,1.0,20130811,SEO,None,G,None,Y,None,1991.0,D/S,M,None,None,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,55.0,2.0,1.0,20160401,None,None,T,O,None,M,1961.0,09302016,M,None,OS,6.666432e+08,93,B2


It looks similar the immigration_data_sample.csv 

Writing the raw data, preferable to an S3 bucket.

In [160]:
# this should idealy be to a S3 bucket 
spark_immi_df.write.parquet("sas_data", 'overwrite')
spark_immi_df = spark.read.parquet("sas_data")
print('Finished writing and reading')

Finished writing and reading


### Reading in Temperature Data to pandas

In [23]:
# reading in the Temperature Data to pandas
pd_temp_df = pd.read_csv('../../data2/GlobalLandTemperaturesByCity.csv')
print('Finished reading Temperature Data to pandas')

Finished reading Temperature Data to pandas


In [162]:
# first look at the Temperature Data
pd_temp_df.head(3)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


### Reading in Demographic Data to pandas

In [209]:
pd_demo_df = pd.read_csv('us-cities-demographics.csv', sep = ';')
print('Finished reading Demographic Data to pandas')

Finished reading Demographic Data to pandas


In [210]:
# first look at the Demographic Data
pd_demo_df.head(3)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759


### Reading in Airport Data to pandas

In [165]:
pd_port_df = pd.read_csv('airport-codes_csv.csv')
print('Finished reading Airport Data to pandas')

Finished reading Airport Data to pandas


In [166]:
# first look at the Airport Data
pd_port_df.head(3)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

### Explore the Immigration Data

In [167]:
pd_immi_df.head(10)

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,NaN,NaN,G,O,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,NaN,G,R,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,NaN,NaN,G,O,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,NaN,G,O,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,NaN,NaN,Z,K,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT
5,721257,1481650.0,2016.0,4.0,577.0,577.0,ATL,20552.0,1.0,GA,20606.0,51.0,2.0,1.0,20160408,NaN,NaN,T,N,NaN,M,1965.0,10072016,M,NaN,DL,7.368526e+08,910,B2
6,1072780,2197173.0,2016.0,4.0,245.0,245.0,SFR,20556.0,1.0,CA,20635.0,48.0,2.0,1.0,20160412,NaN,NaN,T,O,NaN,M,1968.0,10112016,F,NaN,CX,7.863122e+08,870,B2
7,112205,232708.0,2016.0,4.0,113.0,135.0,NYC,20546.0,1.0,NY,20554.0,33.0,2.0,1.0,20160402,NaN,NaN,G,O,NaN,M,1983.0,06302016,F,NaN,BA,5.547449e+10,00117,WT
8,2577162,5227851.0,2016.0,4.0,131.0,131.0,CHI,20572.0,1.0,IL,20575.0,39.0,2.0,1.0,20160428,NaN,NaN,O,O,NaN,M,1977.0,07262016,NaN,NaN,LX,5.941342e+10,00008,WT
9,10930,13213.0,2016.0,4.0,116.0,116.0,LOS,20545.0,1.0,CA,20553.0,35.0,2.0,1.0,20160401,NaN,NaN,O,O,NaN,M,1981.0,06292016,NaN,NaN,AA,5.544979e+10,00109,WT


In [168]:
# count non-NaN for each column
pd_immi_df.count()

Unnamed: 0    1000
cicid         1000
i94yr         1000
i94mon        1000
i94cit        1000
i94res        1000
i94port       1000
arrdate       1000
i94mode       1000
i94addr        941
depdate        951
i94bir        1000
i94visa       1000
count         1000
dtadfile      1000
visapost       382
occup            4
entdepa       1000
entdepd        954
entdepu          0
matflag        954
biryear       1000
dtaddto       1000
gender         859
insnum          35
airline        967
admnum        1000
fltno          992
visatype      1000
dtype: int64

The column abbreviations are explained in the **I94_SAS_Labels_Descriptions.SAS** file and is as follows:

In [169]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

| Abbreviation | Description |
|------------|------------|  
|cicid     |    Immigrant ID number|
|i94yr     |    Year  |
|i94mon    |    Month  |
|i94cit    |    Which city they came from  
i94res     |   Which country they came from
i94port    |   Airport at arrival
 arrdate    |   Arrival Date in the USA  
 i94mode    |   Mode of transportation to the USA  
 i94addr    |   Address state  
 depdate    |   Departure Date from the USA  
 i94bir     |   Age of Respondent in Years  
| i94visa   |   Reasons for immigration: <br /> 1 = Business <br /> 2 = Pleasure <br /> 3 = Student |
 count      |   Used for summary statistics  
 dtadfile   |   Date added to I-94 Files  
 visapost   |   Department of State where where Visa was issued  
 entdepa    |   Admitted or paroled into the U.S.  
 entdepd    |   Departed  
 matflag    |   Match of arrival and departure records  
 biryear    |   Year of birth  
 dtaddto    |   Date to which admitted to U.S. (allowed to stay until)  
 gender     |   Gender  
 airline    |   Airline used to arrive in U.S.  
 admnum     |   Admission Number  
 fltno      |   Flight number of Airline used to arrive in U.S.  
 visatype   |   Class of admission legally admitting the <br /> non-immigrant to temporarily stay in U.S.|

Let's take a look at the **visatype** column. What types are there, and what do they mean?

In [170]:
spark_immi_df.groupBy('visatype').count().orderBy('count', ascending=False).show()

+--------+-------+
|visatype|  count|
+--------+-------+
|      WT|1309059|
|      B2|1117897|
|      WB| 282983|
|      B1| 212410|
|     GMT|  89133|
|      F1|  39016|
|      E2|  19383|
|      CP|  14758|
|      E1|   3743|
|       I|   3176|
|      F2|   2984|
|      M1|   1317|
|      I1|    234|
|     GMB|    150|
|      M2|     49|
|     SBP|     11|
|     CPL|     10|
+--------+-------+



Where 
- WT - Visitors entering for pleasure
- B2 - Visitors ("tourists"): Temporary Visitor for Pleasure or Medical Treatment
- WB - Visitors entering for business purposes
- B1 - Temporary Visa for Business Travelers and domestic servant
etc  
[See more here](https://travel.state.gov/content/travel/en/us-visas/visa-information-resources/all-visa-categories.html)

The **visatype** looks good, so I'll just leave the visa type as it is. 

In [171]:
spark_immi_df.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

Checking the **i94mode** column.

In [172]:
spark_immi_df.createOrReplaceTempView('immi_table')

In [173]:
spark.sql("""
SELECT i94mode, count(*)
FROM immi_table
GROUP BY 1
ORDER BY 2 DESC
""").show()

+-------+--------+
|i94mode|count(1)|
+-------+--------+
|    1.0| 2994505|
|    3.0|   66660|
|    2.0|   26349|
|    9.0|    8560|
|   null|     239|
+-------+--------+



where:

1 = Air <br />
2 = Sea <br />
3 = Land <br />
9 = Not reported  <br />

To keep the data consistent with airports, only the travels by air are kept.

Checking to see if **cicid** can be used as an id

In [174]:
# checking how many rows there are in total
spark.sql("""
SELECT count(*) AS total_amount_rows
FROM immi_table
""").show()

# checking to see how many distinct cicid values
spark.sql("""
SELECT COUNT(DISTINCT(cicid)) AS distinct_rows_cicid
FROM immi_table
""").show()

+-----------------+
|total_amount_rows|
+-----------------+
|          3096313|
+-----------------+

+-------------------+
|distinct_rows_cicid|
+-------------------+
|            3096313|
+-------------------+



Since every row in cicid is distinct, cicid can be used as an id.

The **i94visa** column, or the reason for immigration, is divided into three categories: <br /> 1 = Business <br /> 2 = Pleasure <br /> 3 = Student  

In [175]:
spark.sql("""
SELECT i94visa, count(*)
FROM immi_table
GROUP BY 1
ORDER BY 2 DESC
""").show()

+-------+--------+
|i94visa|count(1)|
+-------+--------+
|    2.0| 2530868|
|    1.0|  522079|
|    3.0|   43366|
+-------+--------+



In [176]:
spark_immi_df.groupby('gender').count().orderBy('count', ascending=False).show()

+------+-------+
|gender|  count|
+------+-------+
|     M|1377224|
|     F|1302743|
|  null| 414269|
|     X|   1610|
|     U|    467|
+------+-------+



Only 'M' and 'F' **genders** will be kept. 

In [177]:
spark_immi_df.groupby('i94bir').count().orderBy('i94bir', ascending=False).show(20)

+------+-----+
|i94bir|count|
+------+-----+
| 114.0|    1|
| 111.0|    1|
| 110.0|    1|
| 109.0|    2|
| 108.0|    2|
| 107.0|    1|
| 105.0|    2|
| 103.0|    1|
| 102.0|    4|
| 101.0|    2|
| 100.0|   24|
|  99.0|   19|
|  98.0|   26|
|  97.0|   52|
|  96.0|   46|
|  95.0|   88|
|  94.0|  104|
|  93.0|  185|
|  92.0|  241|
|  91.0|  319|
+------+-----+
only showing top 20 rows



That a 114 year old has immigrated seems a bit strange, so I'll cap the age at 105. 

The codes in **i94cit** and **i94res** columns needs to be replaced with the corresponding city and country.

The **i94port** column shows the airports the immigrants arrived at, but to keep it compatible with dimension tables I will map the airport to two new columns, **'arrival_city'** and **'arrival_state'**.

### Explore the Temperature Data

In [24]:
# first look at the temperatur data
pd_temp_df.head(10)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
5,1744-04-01,5.788,3.624,Århus,Denmark,57.05N,10.33E
6,1744-05-01,10.644,1.283,Århus,Denmark,57.05N,10.33E
7,1744-06-01,14.051,1.347,Århus,Denmark,57.05N,10.33E
8,1744-07-01,16.082,1.396,Århus,Denmark,57.05N,10.33E
9,1744-08-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [28]:
pd_temp_df['dt'].max()

'2013-09-01'

Looks like 2013 is the last year for the temperature measurment, so this will be filtered by 2013.

In [26]:
# Which countries are in the data
pd_temp_df['Country'].nunique()

159

This study is also only looking at US data, so the other 158 countrys should be dropped.

In [29]:
pd_temp_df = pd_temp_df[pd_temp_df['Country'] == 'United States']

In [30]:
pd_temp_df.count()

dt                               687289
AverageTemperature               661524
AverageTemperatureUncertainty    661524
City                             687289
Country                          687289
Latitude                         687289
Longitude                        687289
dtype: int64

NaN's in 'AverageTemperature' need to be dropped. 

City names letter case should match the letter case for city in the immigration table

### Explore the Demographics Data

In [200]:
pd_demo_df.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [201]:
pd_demo_df.count()

City                      2891
State                     2891
Median Age                2891
Male Population           2888
Female Population         2888
Total Population          2891
Number of Veterans        2878
Foreign-born              2878
Average Household Size    2875
State Code                2891
Race                      2891
Count                     2891
dtype: int64

City names letter case should once again match the letter case for city in the immigration table

In [225]:
pd_demo_df = pd_demo_df.dropna()

In [226]:
pd_demo_df.count()

City                      2875
State                     2875
Median Age                2875
Male Population           2875
Female Population         2875
Total Population          2875
Number of Veterans        2875
Foreign-born              2875
Average Household Size    2875
State Code                2875
Race                      2875
Count                     2875
dtype: int64

Should drop NaN's to keep the data clean.

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

#### Immigration table
- id
- arrival_date
- departure_date
- country_citizenship
- country_residence
- arrival_city
- arrival_state
- age
- gender
- reason_for_immigration
- visatype

#### Temperature table
- city
- date
- average_temperature
- average_temperature_uncertainty
- country

### Demographics table
- 
- 
- 


### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.


The ETL pipeline will consist of the following sections:
- #### Extraction
    - Extraction of csv and sas files
    - Extraction for dictionaries from the I94_SAS_Labels_Descriptions
- #### Transformation
    - Transforming of the Immigration Data
    - Transforming of the Temperature Data
    - Transforming of the Demographic Data
- #### Loading
    - Loading of the data to S3 or locally



#### Extracting data

In [211]:
pd_immigration = pd.read_csv('immigration_data_sample.csv')
pd_temperature = pd.read_csv('../../data2/GlobalLandTemperaturesByCity.csv')
pd_demographic = pd.read_csv('us-cities-demographics.csv', sep = ';')
pd_airports = pd.read_csv('airport-codes_csv.csv')
spark_immigration =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')
print('Finished extracting data')

Finished extracting data


#### Extracting data from I94_SAS_Labels_Descriptions

In [212]:
# reading in the data from the Labels_Descriptions file
i94_sas_label_descriptions_fname = "I94_SAS_Labels_Descriptions.SAS"

with open(i94_sas_label_descriptions_fname) as f:
    Labels_Descriptions = f.readlines()

In [213]:
# creating dictionaries


# making a dictionary with {ports: [city, state]} from the Labels_Descriptions file
re_compiled = re.compile(r"\'(.*)\'.*\'(.*)\'")
valid_ports = {}
for line in Labels_Descriptions[302:962]:
    results = re_compiled.search(line)
    key = results[1]
    value = results[2].split(",")
    if len(value) == 2:
        value[1] = value[1].strip(' ')
        valid_ports[key] = value
    else:
        valid_ports[key] = value
        
        
# making a dictionary with country codes from the Labels_Descriptions file
re_compiled = re.compile(r"\   (.*)\ =.*\'(.*)\'")
country_codes = {}
for line in Labels_Descriptions[9:298]:
    results = re_compiled.search(line)
    country_codes[int(results[1])] = results[2]

### Transforming Data

#### Transformation steps for the Immigration Data
- Date values needs to be transformed to YYYY-MM-DD.
- Port values needs validating. 
- States needs to correspond to the state codes in the Demographic Data.
- Filter i94visa column by mode of transportation to only let travel by air.
- Set the reasons for immigration.
- Filter genders.
- Capping age at 105
- Convert i94res codes to new country_residence column
- Convert i94cit codes to new country_citizenship column

## Transforming the Immigration Data

In [55]:
# How many rows does the original data have?
spark_immigration.count()

3096313

The SAS System represents dates as the [number of days since a reference date](http://v8doc.sas.com/sashtml/ets/chap2/sect5.htm#:~:text=SAS%20date%20values%20are%20written,value%20for%2017%20October%201991.). The reference date, or date zero, used for SAS date values is 1 January 1960. Thus, for example, 3 February 1960 is represented by the SAS System as 33.

In [56]:
# transforming the arrdate column

# create udf to convert SAS date to YYYY-MM-DD
@udf(StringType())
def date_to_isoformat(x):
    """
    argument: SAS system date
    return: datetime
    """
    if x:
        return (datetime(1960, 1, 1).date() + timedelta(x)).isoformat()
    return None

# convert sas date to spark dates in column arrdate and depdate
spark_immigration = spark_immigration.withColumn('arrdate', date_to_isoformat(spark_immigration.arrdate))
spark_immigration = spark_immigration.withColumn('depdate', date_to_isoformat(spark_immigration.depdate))

# test to see if the columns got converted
# spark_immigration.limit(3).select('arrdate', 'depdate').toPandas()

# test to see how many rows dropped in the dropna 
#spark_immigration.count()

[Link to regex page](https://regex101.com/)

In [57]:
# making the arrival_city column
    
    
@udf(StringType())
def port_to_city(port):
    """
    argument: airport
    return: city
    """
    if port in valid_ports:
        return valid_ports[port][0]
    return None
    

spark_immigration = spark_immigration.withColumn('arrival_city', port_to_city(spark_immigration.i94port))

#spark_immigration.limit(3).select('arrival_city').toPandas()

In [58]:
# making the arrival_state column


# if state is same as city, then state == city
@udf(StringType())
def port_to_state(port):
    """
    argument: airport
    return: state
    """
    if port in valid_ports:
        if len(valid_ports[port]) == 2:
            return valid_ports[port][1]
        else:
            return valid_ports[port][0]


spark_immigration = spark_immigration.withColumn('arrival_state', port_to_state(spark_immigration.i94port))

#spark_immigration.limit(3).select('arrival_state').toPandas()

In [59]:
# removing invalid states from i94addr column

# extracting the valid states from Demographic Data
#valid_states = pd_demographics['State Code'].unique().tolist()

# function that checks if a state is in the valid_states list
#@udf(StringType())
#def valid_state(state):
#    if state:
#        if state in valid_states:
#            return state
#    return None


# replace invalid states with NaN
#spark_immigration = spark_immigration.withColumn('i94addr', valid_state(spark_immigration.i94addr))

# removing NaN values from 'i94addr' column 
#spark_immigration = spark_immigration.filter(spark_immigration.i94addr.isNotNull())

# test to see if any states were invalid
#spark_immigration.count()

In [60]:
spark_immigration.createOrReplaceTempView('immigration_table')

In [61]:
# keep only those that travelled by air
spark.sql("""
SELECT *
FROM immigration_table
WHERE i94mode == 1.0
""").createOrReplaceTempView('immigration_table')

In [62]:
# set the reason for immigrating
spark.sql("""
SELECT *, CASE  
               WHEN i94visa = 1 THEN 'Business'
               WHEN i94visa = 2 THEN 'Pleasure'
               WHEN i94visa = 3 THEN 'Student'
               ELSE NULL END as reason_for_immigration
               
FROM immigration_table""").createOrReplaceTempView('immigration_table')

In [63]:
# filter gender
spark.sql("""
SELECT *
FROM immigration_table
WHERE gender IN ('M', 'F')
""").createOrReplaceTempView('immigration_table')

In [64]:
# capping the age at 105
spark.sql("""
SELECT *
FROM immigration_table
WHERE i94bir <= 105.0
""").createOrReplaceTempView('immigration_table')

The age of immigrants look fine, I think. 

In [65]:
# making a temporary table view of the country_codes  

# making a pandas df from country codes, this is the best way to get in a spark dataframe
pd_df_country_codes = pd.DataFrame(list(country_codes.items()),columns = ['country_codes', 'country']) 

# creating a spark dataframe from the pandas dataframe
spark_df_country_codes = spark.createDataFrame(pd_df_country_codes)

spark_df_country_codes.createOrReplaceTempView("country_codes_table")

# test to see if it is correct
#spark.sql("""
#SELECT *
#FROM country_codes_table
#LIMIT 10
#""").show()

In [66]:
# adding a country_residence column with country names from the i94res codes
spark.sql("""
SELECT im.*, cc.country AS country_residence
FROM immigration_table im
JOIN country_codes_table cc
ON im.i94res = cc.country_codes
""").createOrReplaceTempView('immigration_table')

In [67]:
# adding a country_citizenship column with country names from the 94cit codes
spark.sql("""
SELECT im.*, cc.country AS country_citizenship
FROM immigration_table im
JOIN country_codes_table cc
ON im.i94cit = cc.country_codes
""").createOrReplaceTempView('immigration_table')

In [68]:
# overview of the staging table, skip if in a hurry
spark.sql("""
SELECT *
FROM immigration_table
LIMIT 3
""").toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype,arrival_city,arrival_state,reason_for_immigration,country_residence,country_citizenship
0,84587.0,2016.0,4.0,299.0,299.0,SEA,2016-04-01,1.0,None,None,30.0,3.0,1.0,20160401,None,None,T,None,None,None,1986.0,D/S,M,None,CA,6.646275e+08,983,F1,SEATTLE,WA,Student,MONGOLIA,MONGOLIA
1,84590.0,2016.0,4.0,299.0,299.0,SFR,2016-04-01,1.0,WA,2016-08-31,19.0,3.0,1.0,20160401,ULN,None,T,O,None,M,1997.0,D/S,M,None,DL,6.652729e+08,331,F1,SAN FRANCISCO,CA,Student,MONGOLIA,MONGOLIA
2,84591.0,2016.0,4.0,299.0,299.0,SFR,2016-04-01,1.0,CO,None,23.0,3.0,1.0,20160401,ULN,None,G,None,None,None,1993.0,D/S,F,None,UA,9.247326e+10,00138,F1,SAN FRANCISCO,CA,Student,MONGOLIA,MONGOLIA


In [69]:
# creating immigration fact table
fact_immigration_table = spark.sql("""
                                        SELECT 
                                                cicid AS id,
                                                arrdate AS arrival_date,
                                                depdate AS departure_date,
                                                country_citizenship,
                                                country_residence,
                                                arrival_city,
                                                arrival_state,
                                                i94bir AS age,
                                                gender,
                                                reason_for_immigration,
                                                visatype

                                        FROM immigration_table
                                   """)

In [71]:
# counting the number of rows
fact_immigration_table.count()

2263566

In [70]:
# test to see how the fact_immigration_table turned out
fact_immigration_table.limit(5).toPandas()

,id,arrival_date,departure_date,country_citizenship,country_residence,arrival_city,arrival_state,age,gender,reason_for_immigration,visatype
0,84587.0,2016-04-01,None,MONGOLIA,MONGOLIA,SEATTLE,WA,30.0,M,Student,F1
1,84590.0,2016-04-01,2016-08-31,MONGOLIA,MONGOLIA,SAN FRANCISCO,CA,19.0,M,Student,F1
2,84591.0,2016-04-01,None,MONGOLIA,MONGOLIA,SAN FRANCISCO,CA,23.0,F,Student,F1
3,84592.0,2016-04-01,2016-08-31,MONGOLIA,MONGOLIA,SAN FRANCISCO,CA,32.0,F,Pleasure,B2
4,84593.0,2016-04-01,2016-04-20,MONGOLIA,MONGOLIA,SAN FRANCISCO,CA,66.0,F,Pleasure,B2


## Transforming the Temperature Data
#### Temperature Data transforming steps
- Filter for only US using pandas
- Drop rows with NaN's in AverageTemperature column
- Convert dt to datetime
- Only keep records from 2013
- For each city keep only one records per day 
- Make city names upper case to match fact table

In [193]:
# pd_temperature.head(3)

In [172]:
# only keep the temperatures for US
pd_temperature = pd_temperature[pd_temperature['Country'] == 'United States']

In [173]:
# drop rows with NaN in AverageTemperature
pd_temperature = pd_temperature.dropna(how = 'any', subset = ['AverageTemperature'])

In [174]:
# convert dt to datetime
pd_temperature['date'] = pd.to_datetime(pd_temperature.dt)

In [175]:
# only keep temperatures from 2013
pd_temperature = pd_temperature[pd_temperature['date'] >= '2013-01-01']

In [176]:
# only keeping one temperature per day for each city
pd_temperature = pd_temperature.drop_duplicates(['date', 'City'])

In [177]:
pd_temperature.City = pd_temperature['City'].str.upper()

In [181]:
spark_temperature = spark.createDataFrame(pd_temperature)

In [182]:
spark_temperature.createOrReplaceTempView('temperature_table')

In [194]:
#spark.sql("""
#SELECT *
#FROM temperature_table
#LIMIT 3
#""").toPandas()

In [186]:
dim_temperature = spark.sql("""
                                SELECT 
                                        City AS city,
                                        date,
                                        AverageTemperature AS average_temperature,
                                        AverageTemperatureUncertainty AS average_temperature_uncertainty,
                                        Country AS country
                                        
                                FROM temperature_table
                            """)

In [195]:
dim_temperature.count()

2231

In [190]:
dim_temperature.limit(10).toPandas()

,city,date,average_temperature,average_temperature_uncertainty,country
0,ABILENE,2013-01-01,6.320,0.267,United States
1,ABILENE,2013-02-01,8.116,0.222,United States
2,ABILENE,2013-03-01,12.503,0.273,United States
3,ABILENE,2013-04-01,15.753,0.342,United States
4,ABILENE,2013-05-01,22.545,0.125,United States
5,ABILENE,2013-06-01,27.831,0.246,United States
6,ABILENE,2013-07-01,27.421,0.181,United States
7,ABILENE,2013-08-01,28.420,0.362,United States
8,ABILENE,2013-09-01,25.791,1.180,United States
9,AKRON,2013-01-01,-1.086,0.220,United States


## Transforming the Demographic Data
#### Demographic Data transforming steps
- Make city names upper case
- Drop NaN's

In [223]:
# pd_demographics.head()

In [215]:
pd_demographic.City = pd_demographic['City'].str.upper()

In [228]:
pd_demographic = pd_demographic.dropna()

In [229]:
# making a temporary spark table
spark_demographic = spark.createDataFrame(pd_demographic)
spark_demographic.createOrReplaceTempView('demographic_table')

In [230]:
dim_demographic = spark.sql("""
                                SELECT 
                                        City AS city,
                                       `State Code` AS state_code,
                                       `Median Age` AS median_age,
                                       `Male Population` AS male_population,
                                       `Total Population` AS total_population,
                                       `Number of Veterans` AS number_of_veterans,
                                       `Foreign-born` AS foreign_born,
                                       `Average Household Size` AS average_household,
                                        Race AS race,
                                        Count AS count

                                FROM demographic_table
                            """)

In [232]:
dim_demographic.count()

2875

In [233]:
dim_demographic.limit(3).toPandas()

,city,state_code,median_age,male_population,total_population,number_of_veterans,foreign_born,average_household,race,count
0,SILVER SPRING,MD,33.8,40601.0,82463,1562.0,30908.0,2.60,Hispanic or Latino,25924
1,QUINCY,MA,41.0,44129.0,93629,4147.0,32935.0,2.39,White,58723
2,HOOVER,AL,38.5,38040.0,84839,4819.0,8229.0,2.58,Asian,4759


#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.